In [1]:
import pandas as pd 
import requests
import json
from census import Census
import shapefile as shp
import geopandas as gpd
from shapely.geometry import Point, Polygon 
import mplleaflet



In [5]:
natural_deaths=pd.read_csv("../data/input/NCHS_-_Leading_Causes_of_Death__United_States.csv", thousands=',', decimal='.')
natural_deaths.head()

,Year,113 Cause Name,Cause Name,State,Deaths,Age-adjusted Death Rate
0,2017,"Accidents (unintentional injuries) (V01-X59,Y8...",Unintentional injuries,United States,169936,49.4
1,2017,"Accidents (unintentional injuries) (V01-X59,Y8...",Unintentional injuries,Alabama,2703,53.8
2,2017,"Accidents (unintentional injuries) (V01-X59,Y8...",Unintentional injuries,Alaska,436,63.7
3,2017,"Accidents (unintentional injuries) (V01-X59,Y8...",Unintentional injuries,Arizona,4184,56.2
4,2017,"Accidents (unintentional injuries) (V01-X59,Y8...",Unintentional injuries,Arkansas,1625,51.8


In [6]:
#remove us totlas from the data

natural_deaths = natural_deaths.loc[natural_deaths['State'] != "United States"]

natural_deaths.head()

,Year,113 Cause Name,Cause Name,State,Deaths,Age-adjusted Death Rate
1,2017,"Accidents (unintentional injuries) (V01-X59,Y8...",Unintentional injuries,Alabama,2703,53.8
2,2017,"Accidents (unintentional injuries) (V01-X59,Y8...",Unintentional injuries,Alaska,436,63.7
3,2017,"Accidents (unintentional injuries) (V01-X59,Y8...",Unintentional injuries,Arizona,4184,56.2
4,2017,"Accidents (unintentional injuries) (V01-X59,Y8...",Unintentional injuries,Arkansas,1625,51.8
5,2017,"Accidents (unintentional injuries) (V01-X59,Y8...",Unintentional injuries,California,13840,33.2


In [7]:
# #only slect all causes and heart disease 
# natural_deaths = natural_deaths.loc[natural_deaths["Cause Name"].isin(["All causes","Heart disease","Suicide", "Unintentional injuries", "Stroke"])]


# #only select 2010, 2014, 2017
# natural_deaths = natural_deaths.loc[natural_deaths["Year"].isin([2010, 2014, 2017])]

# natural_deaths.head()


In [8]:
# # set the filepath and load in a shapefile
# us_states = gpd.read_file("us_deaths_project/input/tl_2017_us_state/tl_2017_us_state.shp")

# us_states_renamed= us_states.rename(columns={"NAME":"State"})
# us_states_renamed.head()

In [9]:
# #merge the states shapefile dataframe and the natural deaths dataframe 
# merged_geometry_deaths_df = us_states_renamed.merge(natural_deaths, on="State")
# #merged_geometry_deaths_df.head()
# #merged_geometry_deaths_df.count()
# #set index=State
# #merged_geometry_deaths_df2=merged_geometry_deaths_df.set_index("State")

# merged_geometry_deaths_df.head()
# #list(merged_geometry_deaths_df.columns)


In [11]:
#add states population data from : https://www.census.gov/data/datasets/time-series/demo/popest/2010s-state-total.html#par_textimage_500989927
#this is to normalize the death data 
#Annual Estimates of the Resident Population for the United States, Regions, States, and Puerto Rico:
#April 1, 2010 to July 1, 2018

population=pd.read_excel("../data/input/population.xlsx")
#population=population.set_index("State")
population.head()

,State,Year,Population
0,Alabama,2010,4785448
1,Alaska,2010,713906
2,Arizona,2010,6407774
3,Arkansas,2010,2921978
4,California,2010,37320903


In [13]:
#merge the population values and deaths+geometry dataframe 
population_deaths_df = natural_deaths.merge(population, on= ["State","Year"], how="left")
#population_deaths_df.head()

#create a new column that devides the deaths by the statte population to normalize the data 
population_deaths_df[['Deaths', 'Population']] = population_deaths_df[['Deaths', 'Population']].apply(pd.to_numeric)
population_deaths_df["normalized_deaths"]= round((population_deaths_df["Deaths"]/population_deaths_df["Population"])*100000)
population_deaths_df.head()
#population_deaths_df.describe()

,Year,113 Cause Name,Cause Name,State,Deaths,Age-adjusted Death Rate,Population,normalized_deaths
0,2017,"Accidents (unintentional injuries) (V01-X59,Y8...",Unintentional injuries,Alabama,2703,53.8,4875120.0,55.0
1,2017,"Accidents (unintentional injuries) (V01-X59,Y8...",Unintentional injuries,Alaska,436,63.7,739786.0,59.0
2,2017,"Accidents (unintentional injuries) (V01-X59,Y8...",Unintentional injuries,Arizona,4184,56.2,7048876.0,59.0
3,2017,"Accidents (unintentional injuries) (V01-X59,Y8...",Unintentional injuries,Arkansas,1625,51.8,3002997.0,54.0
4,2017,"Accidents (unintentional injuries) (V01-X59,Y8...",Unintentional injuries,California,13840,33.2,39399349.0,35.0


In [15]:
##add on medicare spending by state for 2010 and 2014 in millions 
#https://www.kff.org/medicare/state-indicator/medicare-spending-by-residence/
#?currentTimeframe=0&sortModel=%7B%22colId%22:%22Location%22,%22sort%22:%22asc%22%7D

medicare=pd.read_csv("../data/input/Medicare Spending by State 2010 and 2014.csv")
#population=population.set_index("State")
medicare.head()
medicare.describe()

,Total Medicare Spending by Residence,Year
count,102.000000,102.000000
mean,10481.519608,2012.000000
std,11997.957828,2.009877
min,562.000000,2010.000000
25%,2470.000000,2010.000000
50%,7046.000000,2012.000000
75%,12079.500000,2014.000000
max,64795.000000,2014.000000


In [25]:
#merge the medicare spending values and population deaths dataframe 
population_deaths_medicare_df = population_deaths_df.merge(medicare, on= ["State","Year"], how="left")
population_deaths_medicare_df["normalized_medicare_spending"]= round((population_deaths_medicare_df["Total Medicare Spending by Residence"]/population_deaths_medicare_df["Population"])*100000)
population_deaths_medicare_df.head()



,Year,113 Cause Name,Cause Name,State,Deaths,Age-adjusted Death Rate,Population,normalized_deaths,Total Medicare Spending by Residence,normalized_medicare_spending
0,2017,"Accidents (unintentional injuries) (V01-X59,Y8...",Unintentional injuries,Alabama,2703,53.8,4875120.0,55.0,NaN,NaN
1,2017,"Accidents (unintentional injuries) (V01-X59,Y8...",Unintentional injuries,Alaska,436,63.7,739786.0,59.0,NaN,NaN
2,2017,"Accidents (unintentional injuries) (V01-X59,Y8...",Unintentional injuries,Arizona,4184,56.2,7048876.0,59.0,NaN,NaN
3,2017,"Accidents (unintentional injuries) (V01-X59,Y8...",Unintentional injuries,Arkansas,1625,51.8,3002997.0,54.0,NaN,NaN
4,2017,"Accidents (unintentional injuries) (V01-X59,Y8...",Unintentional injuries,California,13840,33.2,39399349.0,35.0,NaN,NaN


In [26]:
#population_deaths_df.to_csv("us_deaths_project/output/population_deaths_df.csv")


In [28]:
population_deaths_medicare_df.to_json("../data/output/natural_deaths_medicare.json",orient="records")